# Rescan photos scripts

In [1]:
import time

import firebase_admin
import mpmath
from firebase_admin import credentials
from firebase_admin import firestore

cred = credentials.Certificate(r"C:\!Dev\projects\book_detection\keys\avea-biblosphere-210106-firebase-adminsdk-s3bjc-ff2c6ac275 (1).json")
firebase_admin.initialize_app(cred)

db = firestore.client()

#### Not valid photos

In [130]:
# parameter is missed: photo/reporter/bookplace/id/location/url

res = db.collection('photos').where(u'valid', u'==', False).stream()
not_valid = set()
photos = set()
reporters = set()
bookplaces = set()
ids = set()
locations = set()
urls = set()
for doc in res:
    data = doc.to_dict()

    not_valid.add(doc.id)
    if 'photo' not in data:
        photos.add(doc.id)
    if 'reporter' not in data:
        reporters.add(doc.id)
    if 'bookplace' not in data:
        bookplaces.add(doc.id)
    if 'id' not in data:
        ids.add(doc.id)
    if 'location' not in data:
        locations.add(doc.id)
    if 'url' not in data:
        urls.add(doc.id)

    # print(doc.id)

print('not valid:', len(not_valid))
print('parameter is missed:')
print('\tphoto:', len(photos))
print('\treporter:', len(reporters))
print('\tbookplace:', len(bookplaces))
print('\tid:', len(ids))
print('\tlocation:', len(locations))
print('\turl:', len(urls))
#finded

not valid: 2
parameter is missed:
	photo: 0
	reporter: 0
	bookplace: 0
	id: 0
	location: 0
	url: 0


In [88]:
# res = db.collection('photos').where(u'valid', u'==', False).stream()
# for index, doc in enumerate(res):
#     data = doc.to_dict()
#
#     db.collection('photos').document(doc.id).update({'valid': True})
#
#
# # update = {'valid': False}
# # db.collection('photos').document(doc.id).update(update)

In [7]:
res = db.collection('photos').where(u'recognition_stats.record_in_stats', u'==', False).stream()
finded = set()
count = 0
for doc in res:
    finded.add(doc.id)
    # update = {'valid': True}
    # db.collection('photos').document(doc.id).update(update)
    # update = {'valid': False}
    # db.collection('photos').document(doc.id).update(update)


print('records not in stats:', len(finded))
#finded

records not in stats: 0


In [129]:
res = db.collection('photos').where(u'status', u'==', 'failed').stream()
finded = set()
for doc in res:
    # finded.add(doc.id)
    db.collection('photos').document(doc.id).update({'valid': False})
    print(doc.id)
print('status == failed:', len(finded))
#finded

e79opeRXT5jMVHk1mTVJ
rBmM6k1dpRfDRxkJoI49
status == failed: 0


#### Photos without block 'recognition_stats'

In [126]:
res = db.collection('photos').stream()
finded = set()
for doc in res:
    data = doc.to_dict()
    if 'valid' in data and not data['valid']:
        continue
    if 'recognition_stats' in data:
        continue
    finded.add(doc.id)

print('finded', len(finded))
#finded

finded 2


#### View photo params

In [128]:
photo_id = 'e79opeRXT5jMVHk1mTVJ'
doc = db.collection('photos').document(photo_id).get()
data = doc.to_dict()
data

{'status': 'failed',
 'reporter': 'xAw9CoUkO5dOzPv0dVWW2FCY4Ow1',
 'thumbnail': 'https://storage.googleapis.com/biblosphere-210106.appspot.com/thumbnails/UxAw9CoUkO5dOzPv0dVWW2FCY4Ow1:udtsfzd/1613661259996.jpg',
 'id': 'e79opeRXT5jMVHk1mTVJ',
 'bookplace': 'UxAw9CoUkO5dOzPv0dVWW2FCY4Ow1:udtsfzd',
 'url': 'https://firebasestorage.googleapis.com/v0/b/biblosphere-210106.appspot.com/o/images%2FUxAw9CoUkO5dOzPv0dVWW2FCY4Ow1:udtsfzd%2F1613661259996.jpg?alt=media&token=c1e2595d-2bef-40c5-b0d9-29f7874f0577',
 'photo': 'images/UxAw9CoUkO5dOzPv0dVWW2FCY4Ow1:udtsfzd/1613661259996.jpg',
 'count': 0,
 'location': {'geohash': 'udtsfzd2z7bu',
  'geopoint': <google.cloud.firestore_v1._helpers.GeoPoint at 0x13cb8e5b460>}}

#### Rescan one photo

In [127]:
import requests
photo_id = 'e79opeRXT5jMVHk1mTVJ'  # 3W82C9BUmTxM7D0agG2q
url = 'https://us-central1-biblosphere-210106.cloudfunctions.net/rescan_photo'

doc = db.collection('photos').document(photo_id).get()
data = doc.to_dict()

res = requests.post(url, json={'photo_id': photo_id})
json = res.json()
print(json)
print(res.status_code, res.reason)
print('duration:', json['duration'])
print('recognized:', len(json['recognized_books']))
for n, el in enumerate(json['recognized_books'], 1):
    print(f'{n}) {el}')

{'duration': '0:00:22.280705', 'photo_id': 'e79opeRXT5jMVHk1mTVJ', 'recognized_books': []}
200 OK
duration: 0:00:22.280705
recognized: 0


#### Rescan several photos

In [114]:
import requests
url = 'https://us-central1-biblosphere-210106.cloudfunctions.net/rescan_photo'
batch_size = 5

res = db.collection('photos').stream()
count = 1
for doc in res:
    if count > batch_size:
        break

    data = doc.to_dict()
    if 'valid' in data and not data['valid']:
        continue
    # if 'recognition_stats' in data:
    #    continue

    print(f'{count}) Rescaning photo: {doc.id}')

    if not data.keys() >= {'photo', 'reporter', 'bookplace', 'id', 'location', 'url'}:
        print('ERROR: photo/reporter/bookplace/id/location/url parameters missing.', doc.id, '\n')
        update = {'valid': False}
        db.collection('photos').document(doc.id).update(update)
        continue

    old_recognized = data['recognized'] if 'recognized' in data else 0
    res = requests.post(url, json={'photo_id': doc.id})
    json = res.json()
    print(f"duration: {json['duration']}, recognized: {len(json['recognized_books'])}({old_recognized})\n")

    count += 1

1) Rescaning photo: 023YA6RIB7A3boAmZBOq
duration: 0:00:21.805652, recognized: 0(0)

2) Rescaning photo: 03E2LrAHsJucumq0el4Q


KeyboardInterrupt: 

#### Rescan several photos (threads)

In [95]:
import numpy as np
import time

def fill_array(size):
    res = db.collection('photos').stream()
    result = []
    count = 1
    for doc in res:

        if count > batch_size * threads:
            break

        data = doc.to_dict()
        if 'valid' in data and not data['valid']:
            continue
        if 'recognition_stats' in data:
            continue

        if not data.keys() >= {'photo', 'reporter', 'bookplace', 'id', 'location', 'url'}:
            print('ERROR: photo/reporter/bookplace/id/location/url parameters missing.', doc.id, '\n')
            update = {'valid': False}
            db.collection('photos').document(doc.id).update(update)
            continue
        result.append(doc.id)
        count += 1

    return result


def rescan_in_thread(name: str, photo_ids: list):
    url = 'https://us-central1-biblosphere-210106.cloudfunctions.net/rescan_photo'
    for n, id in enumerate(photo_ids):
        print(f'{name}: {n+1}) scanning photo: {id}')
        res = requests.post(url, json={'photo_id': id})
        print(id, res.status_code)



In [121]:
%%time

import requests
from threading import Thread

batch_size = 2
threads = 2

print('--- preparing photo ids ---')
ids = fill_array(batch_size * threads)
print(f'photos: {len(ids)} \n')

print('---starting threads ---')
threads_list = []
for i in range(threads):
    start_ind = i * batch_size
    end_ind = start_ind + batch_size
    thread = Thread(target=rescan_in_thread, args=(f'thread_{i}', ids[start_ind:end_ind]))
    thread.start()
    threads_list.append(thread)

for thread in threads_list:
    thread.join()

print('---finish----')

--- preparing photo ids ---
photos: 3 

---starting threads ---
thread_0: 1) scanning photo: KcgoL01kNZzSGkFpyRRQ
thread_1: 1) scanning photo: rBmM6k1dpRfDRxkJoI49
rBmM6k1dpRfDRxkJoI49 200
KcgoL01kNZzSGkFpyRRQ 200
thread_0: 2) scanning photo: e79opeRXT5jMVHk1mTVJ
e79opeRXT5jMVHk1mTVJ 200
---finish----
Wall time: 56.8 s


In [15]:
from PIL import Image
import requests
from io import BytesIO

photo_id = 'fMwyBKODipSZ15cts9kl'

doc = db.collection('photos').document(photo_id).get()
data = doc.to_dict()
print(data['url'])
response = requests.get(data['url'])
#response.content
img = Image.open(BytesIO(response.content))
img.show()

https://storage.cloud.google.com/biblosphere-210106.appspot.com/images/0000000000000000000000000000/0000000000034.jpg


UnidentifiedImageError: cannot identify image file <_io.BytesIO object at 0x0000016904F59310>

In [17]:
photo_id = 'fMwyBKODipSZ15cts9kl'
update = {'valid': False}
db.collection('photos').document(photo_id).update(update)

update_time {
  seconds: 1642236303
  nanos: 724799000
}